<a href="https://colab.research.google.com/github/gabriellaaileen/ADL/blob/main/Tugas_1_ADL_Bab_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **3.1 Linear Regression**

# ⏪ Vectorization for Speed

In [ ]:
!pip install d2l

In [4]:
%matplotlib inline
import math
import time
import numpy as np
import torch
from d2l import torch as d2l

In [5]:
n = 10000
a = torch.ones(n)
b = torch.ones(n)

In [7]:
class Timer:  #save
    """Record multiple running times."""
    def __init__(self):
        self.times = []
        self.start()

    def start(self):
        """Start the timer."""
        self.tik = time.time()

    def stop(self):
        """Stop the timer and record the time in a list."""
        self.times.append(time.time() - self.tik)
        return self.times[-1]

    def avg(self):
        """Return the average time."""
        return sum(self.times) / len(self.times)

    def sum(self):
        """Return the sum of time."""
        return sum(self.times)

    def cumsum(self):
        """Return the accumulated time."""
        return np.array(self.times).cumsum().tolist()

In [8]:
c = torch.zeros(n)
timer = Timer()
for i in range(n):
    c[i] = a[i] + b[i]  #jumlahin pake loop per elemen
f'{timer.stop():.5f} sec'

'0.12866 sec'

In [9]:
#Vectorizing code often yields order-of-magnitude speedups
timer.start()
d = a + b     #jumlahin pake operator +
f'{timer.stop():.5f} sec'

'0.00021 sec'